## Memoria

La memoria en los modelos de lenguaje (LLMs) permite que estos recuerden información a lo largo de una conversación, simulando la capacidad de "memoria a largo plazo". Esto es útil para mantener un contexto continuo en las interacciones.
¿Cómo funciona la memoria?

La memoria almacena un historial de las interacciones entre el usuario y el modelo, enviándolo como parte del prompt en cada nueva consulta. Sin embargo, debido a la ventana de contexto limitada de los LLMs, se deben gestionar las implementaciones de memoria para optimizar el rendimiento y garantizar respuestas coherentes.
Tipos de memoria en LangChain

- Conversation Buffer: Guarda el historial completo de mensajes.
- Conversation Buffer Window: Retiene solo los mensajes más recientes.
- Conversation Summary: Resume el historial para reducir el contexto.
- Knowledge Graph Memory: Organiza el conocimiento en forma de grafo.

### 1. Conversation Buffer

Esta es la implementación más básica de memoria. Almacena cada mensaje de entrada (prompt) y salida (respuesta) como pares "Human" y "AI". Cada vez que se envía una nueva consulta, todo el historial se adjunta al prompt.
Ventajas

- Permite al modelo recordar toda la conversación.
- Es fácil de implementar.

Desventajas

- Consume rápidamente la ventana de contexto del modelo si la conversación es larga.
- Puede ser ineficiente para interacciones extensas.



In [ ]:
import config

api_key = config.OPENAI_API_KEY

from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

# Configuración del modelo y la memoria
llm = ChatOpenAI(openai_api_key=api_key)  # Modelo de lenguaje
memoria = ConversationBufferMemory()  # Inicializamos la memoria
chatbot = ConversationChain(llm=llm, memory=memoria, verbose=True)  # Creamos el chatbot

# Ejemplo de interacción
chatbot.predict(input="¿Cómo me llamo?")  # El modelo no lo sabrá en esta primera interacción

# Visualización del historial de memoria
memoria.chat_memory.messages


### 2. Conversation Buffer Window Memory

La Conversation Buffer Window Memory es similar al modelo de memoria anterior (Conversation Buffer), pero con una diferencia clave: en lugar de almacenar todo el historial de interacciones, se mantiene solo una "ventana" de mensajes recientes. Esto permite reducir el consumo de contexto, enfocándose en las interacciones más recientes.
Ventajas

- Es más eficiente que la memoria de buffer completa en conversaciones largas.
- Evita que el contexto relevante se pierda entre interacciones pasadas.

Desventajas

- Si la ventana es muy pequeña, el modelo puede perder información importante de interacciones anteriores.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

# Configuración del modelo y la memoria
llm = ChatOpenAI()  # Modelo de lenguaje
memoria = ConversationBufferWindowMemory(k=2)  # La ventana se limita a los últimos 2 mensajes
chatbot = ConversationChain(llm=llm, memory=memoria, verbose=True)  # Crear el chatbot

# Ejemplo de interacción
chatbot.predict(input="¿Cómo me llamo?")  # El modelo no lo sabrá aún

# Visualización del historial en la ventana
memoria.chat_memory.messages


### 3. Conversation Summary Memory

La Conversation Summary Memory es una implementación de memoria que no guarda todo el historial de interacciones ni los mensajes recientes como en los casos anteriores, sino que almacena un resumen de la conversación. Esto reduce significativamente el tamaño del prompt enviado al modelo y es útil para mantener un contexto general sin saturar la ventana de contexto del LLM.
Ventajas

- Ideal para conversaciones largas, ya que evita prompts extensos.
- Proporciona un contexto condensado de la interacción.
- Permite que el modelo conserve información clave sin almacenar todos los detalles.

Desventajas

- La precisión depende de la calidad del resumen generado por el modelo.
- Información específica podría perderse al ser condensada.

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

# Configuración del modelo y la memoria de resumen
llm = ChatOpenAI(stop=["\nHuman"])  # Configuración del modelo
memoria = ConversationSummaryMemory(llm=llm)  # Configuración de memoria basada en resúmenes
chatbot_resumen = ConversationChain(llm=llm, memory=memoria, verbose=True)  # Crear el chatbot

# Ejemplo de interacción
chatbot_resumen.predict(input="¿Qué te gusta a ti de la tecnología?")  # Primera pregunta

# Ver el contenido de la memoria
memoria.chat_memory.messages


### 4. Conversation Knowledge Graph Memory

La Conversation Knowledge Graph Memory almacena el contexto de las interacciones en forma de un grafo de conocimiento. Este enfoque permite identificar relaciones clave y entidades en la conversación, organizándolas de manera estructurada.
Ventajas

- Útil para extraer relaciones entre entidades (personas, objetos, conceptos) en la conversación.
- Facilita un razonamiento más avanzado al organizar la información como triples (sujeto, predicado, objeto).
- Ayuda a capturar conocimiento semántico.

Desventajas

- No es ideal si las interacciones no tienen entidades o relaciones claras.
- Requiere procesamiento adicional para construir y mantener el grafo.



In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

# Importar herramientas adicionales para visualizar grafos
import networkx as nx

# Configuración del modelo y la memoria basada en grafos
llm = ChatOpenAI(stop=["\nHuman"])
memoria = ConversationKGMemory(llm=llm)  # Crear la memoria basada en Knowledge Graph
chatbot_kgm = ConversationChain(llm=llm, memory=memoria, verbose=True)  # Crear el chatbot

# Ejemplo de interacción
chatbot_kgm.predict(input="¿Qué estudió Santi?")  # Primera interacción

# Mostrar los triples extraídos del grafo
print(chatbot_kgm.memory.kg.get_triples())

# Mostrar el historial de la memoria
memoria.chat_memory.messages


### Usando GPT-3 (Instruct models como Davinci, Ada, Curie o Babbage)

Las implementaciones anteriores, basadas en GPT-3.5 Turbo, también pueden adaptarse fácilmente para usar modelos GPT-3 instructivos como Davinci, Ada, Curie o Babbage. Estos modelos están diseñados para instrucciones directas y ofrecen diferentes niveles de rendimiento y costos según la tarea.
Diferencias clave al usar modelos instructivos:

   1. Modelo configurado: Se utiliza la clase OpenAI en lugar de ChatOpenAI.
2. Configuración más sencilla: Estos modelos no manejan mensajes en forma de pares de "Human" y "AI", sino que trabajan directamente con texto plano.
3. Control de temperatura: El parámetro temperature permite ajustar la creatividad o precisión del modelo (valores más bajos son más deterministas).

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Configuración del modelo
llm = OpenAI(temperature=0)  # Usando Davinci, Ada, Curie o Babbage
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()  # Usando memoria de buffer
)

# Ejemplo de interacción
conversation.predict(input="¿Cómo me llamo?")
